### import required libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

### Load Dataset

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset , batch size = 60
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=60, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=60, shuffle=False, num_workers=2)

### Creat model

In [ ]:
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32), 
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64), 
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128), 
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.Dropout(0.2),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(512),  

        )
        self.avgpool = nn.AdaptiveAvgPool2d((2, 2))
        self.classifier = nn.Sequential(
            nn.Linear(512*4, 512),

            nn.ReLU(True),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

### print sample pictures

In [ ]:
from PIL import Image
from IPython.display import display , Markdown
import numpy as np

def index2class(index):

    categories = testset.classes
    
    return(categories[index])

# Convert tensor data to image
def tensor2image(tensor, size):

    numpy_image = tensor.permute(1, 2, 0).numpy()

    numpy_image_scaled = ((numpy_image - numpy_image.min()) / (numpy_image.max() - numpy_image.min()) * 255).astype(np.uint8)

    img = Image.fromarray(numpy_image_scaled)

    img_resized = img.resize(size)
    return img_resized


desired_size = (128, 128)  
for i in range(100,156,6):
    tensor_data = trainset[i][0]
    image = tensor2image(tensor_data, desired_size)
    display(image)
    display(Markdown(str(index2class(trainset[i][1]))))

### Train model

In [ ]:
net = Net()

# Define loss function and optimizer and learning rate
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0005)

# Train the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)
loss_list = list()
for epoch in range(30):  # Adjust number of epochs as needed
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            loss_list.append(running_loss) 
            running_loss = 0.0

print('Finished Training')

### test model

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

### Plot loss_list

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_list)

# Add labels
plt.xlabel('X-axis label')
plt.ylabel('Y-axis label')
plt.title('Plotting a List')

# Show the plot
plt.show()